In [1]:
import pandas as pd
from tables import *
import csv
import numpy as np
import pickle
import time

In [2]:
from tqdm import tqdm_notebook as tqdm

In [3]:
path = '/home/User1/data/self_citations/'

In [4]:
with open(path+'dict_citant.p','rb') as f:
    dict_citant = pickle.load(f)
with open(path+'dict_cite.p','rb') as f:
    dict_cite = pickle.load(f)

In [5]:
with open(path+'dict_cluster_ID.p','rb') as f:
    dict_cluster_ID = pickle.load(f)
with open(path+'dict_cluster_art.p','rb') as f:
    dict_cluster_art = pickle.load(f)

In [6]:
authors = list(dict_cluster_ID.keys())


In [7]:
nb_authors = len(authors)

In [8]:
class self_cits(IsDescription):
    Cluster_ID = UInt32Col()
    ID_Art = UInt32Col()
    other_refs = UInt16Col()
    direct_refs = UInt16Col()
    indirect_refs = UInt16Col()
    direct_cits = UInt16Col()
    indirect_cits = UInt16Col()
    other_cits = UInt16Col()
    

In [9]:
h5file = open_file(path + 'self_citations.h5', mode="w", title="Self Citations")
group = h5file.create_group("/", 'counts')
table = h5file.create_table(group, 'self_cits', self_cits)
h5file.close()

In [10]:
h5file = open_file(path + 'self_citations.h5', mode="a", title="Self Citations")
table = h5file.root.counts.self_cits
row = table.row

In [11]:
articles = list(dict_cluster_art.keys())
nb_articles = len(articles)
arts_idx = np.arange(nb_articles)
np.random.shuffle(arts_idx)

In [ ]:
start_time = time.time()
f = open(path+'progress.txt','w')
for i in range(nb_articles):
    ID_Art = articles[arts_idx[i]]
    authors_art = dict_cluster_art[ID_Art]
    all_arts_from_authors = set()
    for auth_ID in authors_art:
        all_arts_from_authors.update(dict_cluster_ID[auth_ID])
    for ID in authors_art:
        arts_author = set(dict_cluster_ID[ID])
        other_cits = 0
        direct_cits = 0
        indirect_cits = 0
        other_refs = 0
        direct_refs = 0
        indirect_refs = 0

        if ID_Art in dict_citant:
            references = dict_citant[ID_Art]
            for ref in references:
                if ref in arts_author:
                    direct_refs += 1
                elif ref in all_arts_from_authors:
                    indirect_refs += 1
                else:
                    other_refs += 1
        if ID_Art in dict_cite:
            cits = dict_cite[ID_Art]
            for cit in cits:
                if cit in arts_author:
                    direct_cits += 1
                elif cit in all_arts_from_authors:
                    indirect_cits += 1
                else:
                    other_cits += 1

        row['ID_Art'] = ID_Art
        row['Cluster_ID'] = ID
        row['other_cits'] = other_cits
        row['direct_cits'] = direct_cits
        row['indirect_cits'] = indirect_cits
        row['other_refs'] = other_refs
        row['direct_refs'] = direct_refs
        row['indirect_refs'] = indirect_refs
        row.append()
    if i % 1000 == 1 :
        elapsed_time = time.time() - start_time
        elapsed_time_h = np.round(elapsed_time/3600,2)
        time_per_art = elapsed_time/i
        time_left_h = np.round(time_per_art*(nb_articles-i)/3600,2)
        perc = np.round(i/nb_articles*100,3)
        f.write('Progress: {} articles, {}%, Time since start: {}, Time left: {}\n'.format(i,perc,elapsed_time_h,time_left_h))    
        f.flush()
    table.flush()

In [ ]:
f.close()
h5file.close()